In [1]:
import pandas as pd
import gspread
from collections import Counter
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.colab import auth
auth.authenticate_user()  # verify your account to read files which you have access to
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default()) 

In [2]:
# определяем сроки отчета

start_date = '2020-10-02 00:00:00'
end_date = '2020-10-03 23:59:59'

In [3]:
spreadsheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Ycg7zTxds9DZnDvTrFcyNNKuTUxg6Yy6WF0a8Wc02WQ/edit#gid=1167027454') 

def read_sheet(spreadsheet, worksheet):

  sheet =  spreadsheet.get_worksheet(worksheet)
  values = pd.DataFrame(sheet.get_all_records())
  
  return values

transactions = read_sheet(spreadsheet, 1)
clients = read_sheet(spreadsheet, 2)
managers = read_sheet(spreadsheet, 3)
leads = read_sheet(spreadsheet, 4)

In [4]:
#заменяем пустые даты в clients

clients['created_at'].replace(to_replace= '', value='2000-01-01 00:00:00', regex=True, inplace=True)
clients['created_at'].replace(to_replace= '0001-01-01 00:00:00', value='2000-01-01 00:00:00', regex=True, inplace=True)

In [5]:
#меняем формат дат на datetime во всех листах

def convert_date(dt):
    return pd.to_datetime(dt, infer_datetime_format=True)

leads['created_at'] =  convert_date(leads['created_at'])
transactions['created_at'] =  convert_date(transactions['created_at'])
clients['created_at'] =  convert_date(clients['created_at'])

In [6]:
#нашли уникальных менеджеров

def get_unique(managers):
    managers_sheet = pd.DataFrame(managers.manager_id.unique().tolist())  
    managers_sheet.rename(columns={managers_sheet.columns[0]: 'менеджеры'}, inplace=True)
    return managers_sheet.sort_values('менеджеры')

managers_sheet = get_unique(managers)

In [7]:
managers = managers.append({'manager_id' : '00000000-0000-0000-0000-000000000000', 'd_manager' : 'manager #0', 'd_club' : 'club #0'} , ignore_index=True)

In [8]:
leads.rename(columns={ leads.columns[4]: 'd_manager' }, inplace = True)
clients.rename(columns={ clients.columns[2]: 'd_manager' }, inplace = True)

In [9]:
#фильтруем таблички по датам

def filter_by_date(sheet):
    return sheet[(sheet['created_at'] >= start_date) & (sheet['created_at'] <= end_date)]

date_leads = filter_by_date(leads)   
date_transactions = filter_by_date(transactions)
date_clients = filter_by_date(clients)

In [10]:
#посчитали количество заявок

def dt_requests_quantity(date_leads):
    utm_df = pd.DataFrame([Counter(date_leads['d_manager'])]).T 
    utm_df.index.name='менеджеры'
    utm_df.rename(columns={ utm_df.columns[0]: 'количество заявок' }, inplace = True)
    return utm_df

utm_df = dt_requests_quantity(date_leads)

In [11]:
# добавили количество заявок к основной табличке

def merge_table(managers_sheet, to_merge, col_name):
    managers_sheet = managers_sheet.merge(to_merge, on='менеджеры', how='outer').fillna(0)
    managers_sheet[col_name] = managers_sheet[col_name].astype('Int64')
    return managers_sheet

managers_sheet = merge_table(managers_sheet, utm_df, 'количество заявок')

In [12]:
# мусорные заявки (l_client_id, которых нет во вкладке Clients)

def determine_garbage_requests(date_leads, clients):

    garbage_to_count = pd.DataFrame()

    for index, row in date_leads.iterrows():
        if row.l_client_id not in clients.client_id.values:
            garbage_to_count = garbage_to_count.append(row)

    garbage = pd.DataFrame([Counter(garbage_to_count['d_manager'])]).T  # посчитали количество мусорных заявок

    garbage.index.name = 'менеджеры'
    garbage.rename(columns={garbage.columns[0]: 'мусорные заявки'}, inplace=True)

    return garbage


garbage = determine_garbage_requests(date_leads, clients)
managers_sheet = merge_table(managers_sheet, garbage, 'мусорные заявки')  #добавили мусорные заявки к основной табличке

In [13]:
# отсортировали таблички по дате до начала отчета

def filter_before_report(dt):
    return dt[(dt['created_at'] <= start_date)]

before_leads = filter_before_report(leads)
before_transactions = filter_before_report(transactions)
before_clients = filter_before_report(clients)

In [14]:
# новые заявки 

def determine_new_requests(date_leads, transactions, clients):

    new_leads_to_count = pd.DataFrame()

    for index, row in date_leads.iterrows():
      if row['l_client_id'] not in before_clients.client_id.values and row['l_client_id'] not in before_transactions.l_client_id.values:
        new_leads_to_count = new_leads_to_count.append(row)
    new_leads_to_count.head()  # собрали их в одном df


    new_leads = pd.DataFrame([Counter(new_leads_to_count['d_manager'])]).T  # посчитали количество новых заявок
    new_leads.index.name = 'менеджеры'
    new_leads.rename(columns={new_leads.columns[0]: 'новые заявки'}, inplace=True)

    return new_leads

new_leads = determine_new_requests(date_leads, transactions, clients)
managers_sheet = merge_table(managers_sheet, new_leads, 'новые заявки')

In [15]:
# количество покупателей (кто купил в течение недели после заявки)

leads_with_transactions = pd.DataFrame()
profit_transactions = pd.DataFrame()

for l_index, l_row in date_leads.iterrows():
  if l_row['l_client_id'] in transactions['l_client_id'].values:
    for t_index, t_row in transactions.iterrows():
      if t_row['l_client_id'] == l_row['l_client_id'] and l_row['created_at'] <= t_row['created_at'] <= l_row['created_at'] + pd.DateOffset(days=7):
        profit_transactions = profit_transactions.append(t_row)  # транзакции для подсчета дохода новых покупателей 
        leads_with_transactions = leads_with_transactions.append(l_row)

leads_with_transactions = leads_with_transactions.drop_duplicates(subset = ['l_client_id'])

customers = pd.DataFrame([Counter(leads_with_transactions['d_manager'])]).T 
customers.index.name='менеджеры'
customers.rename(columns={ customers.columns[0]: 'количество покупателей' }, inplace = True)

managers_sheet = merge_table(managers_sheet, customers, 'количество покупателей')

In [16]:
# количество новых покупателей (кто купил в течение недели после заявки, и не покупал раньше)

leads_new_customers = pd.DataFrame()

for index, row in leads_with_transactions.iterrows():
  if row['l_client_id'] not in before_transactions['l_client_id'].values:
    leads_new_customers = leads_new_customers.append(row)

new_customers = pd.DataFrame([Counter(leads_new_customers['d_manager'])]).T 
new_customers.index.name='менеджеры'
new_customers.rename(columns={ new_customers.columns[0]: 'количество новых покупателей' }, inplace = True)

managers_sheet = merge_table(managers_sheet, new_customers, 'количество новых покупателей')

In [17]:
# доход от покупок новых покупателей

def profit_new_customers(profit_transactions, leads_new_customers):

  for index, row in profit_transactions.iterrows():
    if row['l_client_id'] not in leads_new_customers['l_client_id'].values:
      profit_transactions = profit_transactions.drop(index)  # откинули транзакции покупателей, которые не были новыми

  leads_new_customers = leads_new_customers.groupby('d_manager')['l_client_id'].apply(list).reset_index(name='l_client_id')
  leads_new_customers['доходы от новых покупателей'] = 0

  for l_index, l_row in leads_new_customers.iterrows():
    purchase = []
    for i in l_row['l_client_id']:
      for t_index, t_row in profit_transactions.iterrows():
        if i == t_row['l_client_id']:
          purchase.append(int(t_row['m_real_amount']))
    total = sum(purchase)
    leads_new_customers['доходы от новых покупателей'][l_index] = total

  leads_new_customers.rename(columns={ leads_new_customers.columns[0]: 'менеджеры' }, inplace = True)
  leads_new_customers = leads_new_customers.drop(columns = 'l_client_id')

  return leads_new_customers

profit_new_customers = profit_new_customers(profit_transactions, leads_new_customers)
managers_sheet = merge_table(managers_sheet, profit_new_customers, 'доходы от новых покупателей')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
# заменили id менеджеров на клубы

for l_index, l_row in managers_sheet.iterrows():
  for m_index, m_row in managers.iterrows():
    if l_row['менеджеры'] == m_row['manager_id']:
      managers_sheet['менеджеры'].replace( {l_row['менеджеры']:m_row['d_manager'] }, inplace = True )


managers_sheet

,менеджеры,количество заявок,мусорные заявки,новые заявки,количество покупателей,количество новых покупателей,доходы от новых покупателей
0,manager #8,13,2,6,2,2,156730
1,manager #11,6,0,2,1,0,0
2,manager #12,2,0,2,0,0,0
3,manager #10,2,0,0,0,0,0
4,manager #9,2,0,1,0,0,0
5,manager #5,0,0,0,0,0,0
6,manager #7,0,0,0,0,0,0
7,manager #2,5,0,3,0,0,0
8,manager #13,1,0,1,0,0,0
9,manager #1,3,0,0,0,0,0


In [20]:
final_report = gc.open_by_url('https://docs.google.com/spreadsheets/d/1UWxqIwQXV3uRRQ21tY-B3Ms-S56aE7RItWMCfeLjtrw/edit#gid=0') 
managers_table = final_report.add_worksheet(title= 'менеджеры', rows='100', cols='100')
set_with_dataframe(managers_table, managers_sheet)